## Manipulating files with AWK and grep
### Bioinformatics Coffee Hour - April 20, 2021
### Author: Danielle Khost



## Manipulating files with grep
--------
In this lesson, we will explore how we can use basic command line tools to parse, subset, and rearrange common file types you will come across in bioinformatics.

**grep** is a powerful command-line search tools that is included as part of most Unix-like systems. It is one of the most useful tools in bioinformatics! At the most basic level, grep searches for a string of characters that match a pattern and will print lines containing a match. Basic syntax is:  

`grep 'pattern' file_to_search`  

By default, grep will match any part of the string, so for example:

In [ ]:
grep 'dog' data/sample.txt

This will match the line, as will:

In [ ]:
grep 'do' data/sample.txt

It is important to be mindful of partial matching, as you can end up selecting lines that you do not intend! This is just the most basic use of grep however, and there are a huge number of ways to modify its behavior. Here are just a few useful examples:

`grep -w` matches *entire words*.
- So in the above example:  
`grep -w 'dog' data/sample.txt` would match the string, but `grep -w 'do' data/sample.txt` would not.

`grep -i` allows case-insensitive matches
- In the above example, `grep -i 'DOG'` would still match the line

`grep -v` *inverts*, returning lines that *do not* match the pattern.

`grep -o` returns only the matching words, not the entire line.

`grep -c` counts the number of lines that match the pattern.
- Equivalent to `grep 'pattern' file | wc -l`

Print lines before/after a match:
- `grep -A [n]` returns matching line and *n* lines after match
- `grep -B [n]` returns matching line and *n* lines before match
- `grep -C [n]` returns matching line and *n* lines before *and* after match

---

The real strength of grep is that we can combine these different arguments together. For instance, let's say we have a single-line fasta file and we want to quickly pull out the X chromosome:

In [ ]:
grep -w -A 1 '>X' data/dmel-subset-chromosomes.fasta

This code will match line that start with ">X" (using the -w flag to match whole word only) and will also print the line *after* the match as well using the `-A 1` flag (i.e. the sequence line of our fasta file; note this will not work if fasta file is multi-line!).

There are many other functions of grep! When in doubt, remember you can check the help page using `man grep`...or just by using google :)

### Pattern matching with regular expressions
Regular expressions, aka "regex", are patterns that describe sets of strings. In other words, they allow you to match complex patterns with grep, not just exact matches. Regex is extremely powerful, but can also get (very) complicated, so we'll just stick to a few basic uses.

Regex has certain meta-characters that are reserved for special uses:

```
^: matches pattern at start of string
$: matches pattern at end of string
.: matches any character except new lines
[]: matches any of enclose characters
[^]: matches any characters *except* ones enclosed (note: is different from ^)
\: "escapes" meta-characters, allows literal matching
```

Note that if we want to match any of these special characters literally, we would need to use a "\\" to escape it first. So if we wanted to literally match a period (".") character we would run:

In [ ]:
grep '\.' data/Homo_sapiens.GRCh38.subset.gff3 | head

One example of how regex can come in handy is using the ^ special character to quickly count how many sequences are in a FASTA file, which we would do as follows:

In [ ]:
grep -c '^>' data/dmel-subset-chromosomes.fasta

This command matches lines in the FASTA file that start with a ">" character, i.e. the header lines, and uses the -c argument to count how many matches!

---

## What is awk?
Invented in the 1970's, [awk](https://en.wikipedia.org/wiki/AWK) is a scripting language included in most Unix-like operating systems. It specializes in one-liner programs and manipulating text files.

In many cases, if you're parsing information from a text file (such as a [BED](https://en.wikipedia.org/wiki/BED_(file_format)) file, [FASTA](https://en.wikipedia.org/wiki/FASTA) file, etc.), you could write a Python script...or you could do it with awk in a single line!

### Syntax
awk scripts are organized as:

`awk 'pattern { action; other action }' file`

Meaning that every time that the pattern is true, awk will execute the action in the brackets.
If no pattern is specified, the action will be taken for every line in the input file, e.g. the following command prints every line:

In [ ]:
awk '{print}' data/hg38.genome | head

The two most important patterns are `BEGIN` and `END`, which tell the action to take place before any lines are read and after the last line.

In [ ]:
awk 'BEGIN{sum=0} {sum+=1} END {print sum}' data/hg38.genome

 The above line sets a variable at the start of the script, adds 1 to it every line, then prints its value at the end.

If a variable hasn't been initialized, it is treated as 0 in numeric expressions, and an empty string in string expressions&mdash;awk will not print an error!
So the following awk script also prints the number of lines in the file data/hg38.genome:

In [ ]:
awk '{sum+=1} END {print sum}' data/hg38.genome

### Input and output
Input to awk is split into **records** and **fields**.
- By default, **records** are separated by newline character, i.e # of records = # of lines in input file
- Each record is subdivided into **fields**, i.e. columns, as determined by the field separator (see below)

There are several important built-in variable in awk.
The fields (columns) of each record are referred to by `$number`, so the first column would be `$1`, second would be `$2`, etc. `$0` refers to the entire record.

So to print the second column of each line in the file, we'd use:

In [ ]:
awk '{print $2}' data/hg38.genome | head

And if we wanted to print the second then the first:

In [ ]:
awk '{print $2,$1}' data/hg38.genome | head

Note that when the different fields are separated with commas in the `print` statement, they are joined by the output field separator (the **OFS** variable, described below), which is by default a space.
If the comma is omitted between fields (e.g., `awk '{print $2 $1}'`, they are concatenated without a separator.

We can also print strings using using quotation marks:

In [ ]:
awk '{print "First column:" $1}' data/hg38.genome | head

Which for every line of the file will print the text "First column:" followed by the value in the first field.

---
awk has several other built-in variables that are very useful for parsing text, including:

|  |   |
---|---|
| **FS** | field separator (default: white space) |
| **OFS** |  output field separator, i.e. what character separates fields when printing|
| **RS** | record separator, i.e. what character records are split on (default: new line) |
| **ORS** | output record separator |
| **NR** | number of records in input (# lines by default) |

Assigning to a field causes the entire record ($0) to be recomputed using **OFS**. We can use this to convert between file formats, e.g. make a comma-separated text file into a tab-separated file. First, let's look at the first few lines on our comma-separated file using `head`:

In [ ]:
head data/enhancers.csv

Now let's convert the file:

In [ ]:
awk 'BEGIN{FS="," ; OFS="\t"} {$1 = $1; print $0}' data/enhancers.csv | head

### Conditionals and pattern matching
Like other programming languages, awk allows conditional matching with if/else statements.

`awk '{if(condition) action; else other_action}'`

awk uses the following conditional operators:

| | |
|-|-|
|==|equal to|
|!=|not equal to|
|>|greater than|
|>=|greater than or equal to|
|<|less than|
|<=|less than or equal to|
|&&|AND|
| \|\| |OR|
| ! | NOT |

In addition, awk also supports string matching using regular expressions, using the following expressions:

| | |
|-|-|
|\~|matches|
|!~|does not match|

For string matching, the pattern being matched must be enclosed by slashes, like so:

`awk '{if($1 ~ /pattern/) print}'`

Note that if an action isn't specified, the default action is `{print}`, so the previous awk command is equivalent to the following, which specifies only a pattern expression:

`awk '$1 ~ /pattern/'`

---
### Example uses:

- Count number of sequences in a FASTQ file:

In [ ]:
awk 'END{print NR/4}' data/example.fastq

**Note**: this is technically safer than using grep, as you don't have to worry about accidentally counting the quality line.

- Only print annotations on a specific scaffold (chr1) that fall between 1Mb and 2Mb from a BED annotation file (a common file type that lists genomic coordinates of certain features). First let's look at the first few lines of the file using `head`:

In [ ]:
head data/Homo_sapiens_ucscGenes.subset.bed

Now subset the file:

In [ ]:
awk 'BEGIN{FS="\t";OFS="\t"} {if($1 == "chr1" && $2 >=1000000 && $2 <= 2000000) print}' data/Homo_sapiens_ucscGenes.subset.bed | head

**Note**: when we specify that we only want annotations from chr1, we're using exact match (`== "chr1"`) and not pattern match (`~ /chr1/`)...why is this??

- Another common type of annotation file is a Genome Feature File (GFF), which also lists coordinates. Let's look at the first few lines of the file:

In [ ]:
head data/Homo_sapiens.GRCh38.subset.gff3

Now let's subset the file and only print lines that match the string "exon" in their third column:

In [ ]:
awk 'BEGIN{FS="\t"} {if($3 ~ /exon/) print $0}' data/Homo_sapiens.GRCh38.subset.gff3 | head

- Convert from GFF (genome feature file) to BED file 

In [ ]:
grep -v '^#' data/Homo_sapiens.GRCh38.subset.gff3 | awk 'BEGIN{FS="\t"; OFS="\t"} {print $1,$4-1,$5}' | head

**Note**: Annoyingly, BED and GFF files have different coordinate systems, i.e. BED start coordinate is 0 based, half-open, GFF is 1-based inclusive! Also, we are first using grep to skip the header lines in the GFF file.

Alternatively, you could do the whole thing with only awk, no grep required:

In [ ]:
awk 'BEGIN{FS="\t"; OFS="\t"} !/^#/ {print $1,$4-1,$5}' data/Homo_sapiens.GRCh38.subset.gff3 | head

With this command, `!/^#/` is a pattern (like `BEGIN` or `END`) that tell awk to execute the print statement when the start of the line does not match a `#`. Use whichever makes the most sense to you!

### Practice
Using awk:

* Pull out only the CDS annotations (i.e. has "CDS" in the 3rd column) from the GFF file data/Homo_sapiens.GRCh38.subset.gff3 and output them in BED format

*Try it*

In [ ]:
awk '...' data/Homo_sapiens.GRCh38.subset.gff3 | head

*Solution*

In [ ]:
awk 'BEGIN{FS="\t"; OFS="\t"} {if($3 ~ /CDS/) print $1,$4-1,$5}' data/Homo_sapiens.GRCh38.subset.gff3

- Calculate the average length of gene annotations from the file data/Homo_sapiens_ucscGenes.subset.bed

*Try it*

In [ ]:
awk '...' data/Homo_sapiens_ucscGenes.subset.bed

*Solution*

In [ ]:
awk 'BEGIN{FS="\t"; sum=0} {len=$3-$2; sum=sum+len} END{print sum/NR}' data/Homo_sapiens_ucscGenes.subset.bed